<a href="https://colab.research.google.com/github/BrunoLad/tcc-ufabc/blob/main/tcc_ufabc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Monta o google drive para permitir a leitura dos dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalando o Stanza - Wrapper do Stanford CoreNLP para python.

> É possível instalar o próprio NLTK que possui referência para o Stanford CoreNLP, porém o stanza é o [recomendado](https://stanfordnlp.github.io/CoreNLP/other-languages.html#official-stanza-package-by-the-stanford-nlp-group).

In [2]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 7.0 MB/s 
     |████████████████████████████████| 4.4 MB 50.4 MB/s 
     |████████████████████████████████| 175 kB 61.3 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 596 kB 22.2 MB/s 
     |████████████████████████████████| 101 kB 14.9 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=fea1026a946d1d8732b749da5a328a355f001c16764d9f07c310726bc077c6d8
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Realiza o filtro dos livros que são de ficção
O filtro ocorre da seguinte forma:
Uma classe BookInfo é definida, que contém as propriedades do nome do arquivo (para obtenção do texto para análise posterior), uma ref pra class autor que contém nome e gênero do autor (para categorização posterior), título da obra (para identificação), id do livro (para obtenção dos metadados do livro e filtragem dos livros que são de ficção)

In [ ]:
import os
import regex
import json
from enum import Enum

class BookInfo:
  def __init__(self, file_name):
    pattern = r'(.+)(?:___)(.+)\.txt'
    book_regex = re.compile(pattern)
    self.filename = file_name
    self.author = Author(book_regex.match(file_name).group(1))
    self.title = book_regex.match(file_name).group(2)
    self.bookid = None
    self.metadata = None

class Author:
  def __init__(self, name):
    self.name = name
    self.gender = None
  
  def add_gender(self, gender):
    self.gender = gender.value

class Gender(Enum):
  FEMALE = 'F'
  MALE = 'M'

# Necessário tratar os casos em que o nome do autor encontra-se na
# linha posterior ao título da obra
def extract_line_from_file(file_name, book):
  with open(file_name, 'r') as read_obj:
    for line in read_obj:
      if book.title in line and book.author in line:
        return line
  return ''

if __name__ == '__main__':
  BOOKS_HOME = '/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/txt'
  gutenberg_metadata_file = open('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/gutenberg-metadata.json')
  gutenberg_metadata = json.load(gutenberg_metadata_file)

  pattern = r'[^_]*$'
  digitPattern = r'\d+'
  re_bookId = re.compile(digitPattern)
  regex = re.compile(pattern, re.MULTILINE)
  fileList = os.listdir(BOOKS_HOME)
  newBooks = map(lambda book: BookInfo(book), fileList)
  newBooksList = list(newBooks)
  print(len(newBooksList))
  
  for book in newBooksList:
    line = extract_line_from_file('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/GUTINDEX.ALL14062022.txt', book)
    match = re_bookId.search(line)
    if match:
      book.bookid = match.group()
      
  filteredBooks = list(filter(lambda book: book.bookid is not None, newBooksList))
  fiction_books = filter(lambda book: book.bookid in gutenberg_metadata and re.search(r'[fF]iction', json.dumps(gutenberg_metadata[book.bookid]['subject'])), filteredBooks)
  fiction_books_list = list(fiction_books)
  print(len(fiction_books_list))

  for book in fiction_books_list:
    print('{0}: {1} by {2}'.format(book.bookid, book.title, book.author))

  

In [3]:
import os
from enum import Enum
import re

class BookInfo:
  def __init__(self, file_name):
    pattern = r'(.+)(?:___)(.+)\.txt'
    book_regex = re.compile(pattern)
    self.filename = file_name
    self.author = Author(book_regex.match(file_name).group(1))
    self.title = book_regex.match(file_name).group(2)
    self.bookid = None
    self.metadata = None

class Author:
  def __init__(self, name):
    self.name = name
    self.gender = None
  
  def add_gender(self, gender):
    self.gender = gender.value

class Gender(Enum):
  FEMALE = 'F'
  MALE = 'M'

if __name__ == '__main__':
  fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/cleaned')
  newBooks = map(lambda book: BookInfo(book), fileList)
  fiction_books_list = list(newBooks)

## Copia os livros de ficção para uma nova pasta para futuras manipulações

In [ ]:
from shutil import copyfile
from pathlib import Path

FICTION_BOOKS_HOME = '/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/fiction'
fiction_path = Path(FICTION_BOOKS_HOME)

# Remove a pasta e todo seu conteúdo caso ela já existia
# E Cria a pasta do zero
if fiction_path.exists():
  [f.unlink() for f in fiction_path.itedir() if f.is_file()]
  fiction_path.rmdir()

fiction_path.mkdir(parents = True, exist_ok = True)

for book in fiction_books_list:
  copyfile(BOOKS_HOME + '/' + book.filename, FICTION_BOOKS_HOME + '/' + book.filename)

894
Philip Kindred Dick___The Eyes Have It.txt
Philip Kindred Dick___Beyond Lies the Wub.txt
Philip Kindred Dick___Second Variety.txt
Philip Kindred Dick___The Crystal Crypt.txt
Rafael Sabatini___Bardelys the Magnificent.txt
Rafael Sabatini___Captain Blood.txt
Rafael Sabatini___Mistress Wilding.txt
R M Ballantyne___Under the Waves.txt
Rafael Sabatini___Love-at-Arms.txt
Rafael Sabatini___St. Martin's Summer.txt
Rafael Sabatini___Scaramouche.txt
Rafael Sabatini___The Snare.txt
Rafael Sabatini___The Trampling of the Lilies.txt
Rafael Sabatini___The Historical Nights' Entertainment, First Series.txt
Rafael Sabatini___The Tavern Knight.txt
Rafael Sabatini___The Lion's Skin.txt
Robert Louis Stevenson___David Balfour, Second Part.txt
Robert Louis Stevenson___Catriona.txt
Robert Louis Stevenson___Island Nights' Entertainments.txt
Robert Louis Stevenson___Fables.txt
Robert Louis Stevenson___Kidnapped.txt
Robert Louis Stevenson___New Arabian Nights.txt
Robert Louis Stevenson___The Merry Men.txt


## Limpa os livros
Faz várias limpezas nos livros obtidos inplace (no mesmo arquivo que foi aberto e já inserido no diretório de livros de ficção), tanto a fim de aumentar a acurácia, quanto a fim de diminuir o tamanho dos livros com o intuito de otimizar o tempo de análise.

As limpezas efetuadas são:
- Transforma todos os whitespaces no mesmo tipo ' '
- Remove todos os dados antes do primeiro capitulos, nos livros que possuem o padrao:
  - Chapter [1I]
- Pega todo o conteúdo depois do separador 'by nomeAutor'
- Subsitui algarismos romanos por outra delimitador mais fácil de remover e depois remove linhas que iniciem com este delimitador que foi inserido
- Transforma o sinal tironiano num 'and' normal da língua inglesa
- Remove o nome do autor caso encontrado no livro
- Remove o título da obra caso encontrado no livro
- Remove espaços e quebras de linha em excesso

In [ ]:
import os
import re

FICTION_BOOKS_HOME = '/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/cleaned'
roman_pattern = """(M{1,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|C?D|D?C{1,3})(XC|XL|L?X{0,3})\
(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|X?L|L?X{1,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})\
(XC|XL|L?X{0,3})(IX|I?V|V?I{1,3}))"""

# Atualiza o livro (objeto do tipo arquivo, aberto na localização daquele livro)
# com o content passado
def update_book_data(book_file, content):
  book_file.seek(0)
  book_file.write(content)
  book_file.truncate()

# Elimina "impurezas" do corpo do livro
def clean_book(book):
  # with open(f'{FICTION_BOOKS_HOME}/{book.filename}', r) as bookfile:
  with open(f'{FICTION_BOOKS_HOME}/{book.filename}', 'r+') as bookfile:
    book_content = bookfile.read()
    
    # Transforma todos os whitespaces no mesmo tipo
    book_content = re.sub(r'\s', ' ', book_content)

    # Remove todos os dados antes do primeiro capitulos, nos livros que possuem o padrao
    # Chapter [1I]
    #book_content = re.sub(r"contents(\s*\w\.*'*)*\s{2}", '', book_content, flags = re.IGNORECASE | re.MULTILINE)
    content = re.split(r'^chapter [I1].*$', book_content, flags = re.IGNORECASE | re.MULTILINE)
    book_content = content[1] if len(content) > 1 else content[0]

    # Pega todo o conteudo depois do separador 'by {authorName}'
    content = re.split(fr'by\s+{book.author.name}', book_content, flags = re.IGNORECASE | re.MULTILINE)
    book_content = content[1] if len(content) > 1 else content[0]
    
    # Remove Illustrations indicator
    book_content = re.sub(r'\[Illustration\w*\]', '', book_content, flags = re.IGNORECASE | re.MULTILINE)
    
    # Subsitui algarismos romanos por outra delimitador mais fácil de remover
    # Depois remova linhas que iniciem com este delimitador que foi inserido
    book_content = re.sub(fr'\s*^{roman_pattern}\b', '%%%%%', book_content, flags = re.IGNORECASE | re.MULTILINE)
    book_content = re.sub(r'^(%{5}.*)$', '', book_content, flags = re.IGNORECASE | re.MULTILINE)


    # Transforma o sinal tironiano num 'and' normal da língua inglesa
    # para facilitar o processamento do texto
    book_content = re.sub(r'&', 'and', book_content)
    # Remove o nome do autor caso encontrado no livro
    book_content = re.sub(book.author.name, '', book_content, re.IGNORECASE)
    # Remove o título da obra caso encontrado no livro
    book_content = re.sub(book.title, '', book_content, re.IGNORECASE)

    # Remove espaços e quebras de linha em excesso
    book_content = re.sub(r'\s+', ' ', book_content)

    update_book_data(bookfile, book_content)


from enum import Enum

class BookInfo:
  def __init__(self, file_name):
    pattern = r'(.+)(?:___)(.+)\.txt'
    book_regex = re.compile(pattern)
    self.filename = file_name
    self.author = Author(book_regex.match(file_name).group(1))
    self.title = book_regex.match(file_name).group(2)
    self.bookid = None
    self.metadata = None

class Author:
  def __init__(self, name):
    self.name = name
    self.gender = None
  
  def add_gender(self, gender):
    self.gender = gender.value

class Gender(Enum):
  FEMALE = 'F'
  MALE = 'M'

fileList = os.listdir(f'{FICTION_BOOKS_HOME}/')
newBooks = map(lambda book: BookInfo(book), fileList)
cleaned_books_list = list(newBooks)
print(len(cleaned_books_list))
  
for book in cleaned_books_list:
  clean_book(book)



894


## Faz [anotações](https://stanfordnlp.github.io/CoreNLP/annotators.html) suportadas pelo StanfordCoreNLP na base de livros:


In [ ]:
import os
import stanza
from stanza.server import CoreNLPClient

# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
os.environ["CORENLP_HOME"] = corenlp_dir

with open('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/cleaned/' + fiction_books_list[0].filename, 'r') as file:
  data = file.read()

print('{0} by {1}'.format(fiction_books_list[0].title, fiction_books_list[0].author.name))

with CoreNLPClient(
    #annotators = ['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'],
    annotators = ['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'depparse', 'coref', 'openie'],
    #properties = {'annotators': 'coref', 'coref.algorithm': 'statistical'},
    be_quiet = False,
    timeout = 150000000,
    memory = '10G',
    endpoint='http://localhost:9001',
    max_char_length = 2000000) as client:
  ann = client.annotate(data)

2022-03-30 00:04:57 WARNING: Directory ./corenlp already exists. Please install CoreNLP to a new directory.
2022-03-30 00:04:57 INFO: Writing properties to tmp file: corenlp_server-1c3471169beb4743.props
2022-03-30 00:04:57 INFO: Starting server with command: java -Xmx10G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 150000000 -threads 5 -maxCharLength 2000000 -quiet False -serverProperties corenlp_server-1c3471169beb4743.props -annotators tokenize,ssplit,pos,lemma,ner,depparse,coref -preload -outputFormat serialized


Crome Yellow by Aldous Huxley


In [ ]:
sentence = ann.sentence[0]
  
# print(sentence.mentions[0])
print(len(ann.sentence))
# print(ann.sentence[2].mentions)

for count, sentence in enumerate(ann.sentence):
  print('Id {0}: {1}'.format(count, sentence.mentions))


#print(ann.sentence)

A saída de streaming foi truncada nas últimas 5000 linhas.
tokenStartInSentenceInclusive: 65214
tokenEndInSentenceExclusive: 65217
ner: "DATE"
normalizedNER: "OFFSET PT1M"
entityType: "DATE"
timex {
  altValue: "OFFSET PT1M"
  text: "A minute later"
  type: "DATE"
  tid: "t261"
}
entityMentionIndex: 5470
canonicalEntityMentionIndex: 5470
entityMentionText: "A minute later"
, sentenceIndex: 3745
tokenStartInSentenceInclusive: 65182
tokenEndInSentenceExclusive: 65183
ner: "PERSON"
entityType: "PERSON"
gender: "MALE"
entityMentionIndex: 5471
canonicalEntityMentionIndex: 5101
entityMentionText: "he"
, sentenceIndex: 3745
tokenStartInSentenceInclusive: 65217
tokenEndInSentenceExclusive: 65218
ner: "PERSON"
entityType: "PERSON"
gender: "MALE"
entityMentionIndex: 5472
canonicalEntityMentionIndex: 5101
entityMentionText: "he"
]
Id 3746: []
Id 3747: [sentenceIndex: 3747
tokenStartInSentenceInclusive: 65244
tokenEndInSentenceExclusive: 65245
ner: "TITLE"
entityType: "TITLE"
entityMentionIndex: 5

In [ ]:
print(ann.sentence[0])

token {
  word: "CHAPTER"
  pos: "NN"
  value: "CHAPTER"
  before: " "
  after: " "
  originalText: "CHAPTER"
  ner: "O"
  lemma: "chapter"
  beginChar: 1
  endChar: 8
  utterance: 0
  speaker: "PER0"
  tokenBeginIndex: 0
  tokenEndIndex: 1
  hasXmlContext: false
  isNewline: false
  coarseNER: "O"
  fineGrainedNER: "O"
  corefMentionIndex: 0
  nerLabelProbs: "O=0.9890408271949985"
}
token {
  word: "I."
  pos: "NNP"
  value: "I."
  before: " "
  after: " "
  originalText: "I."
  ner: "O"
  lemma: "I."
  beginChar: 9
  endChar: 11
  utterance: 0
  speaker: "PER0"
  tokenBeginIndex: 1
  tokenEndIndex: 2
  hasXmlContext: false
  isNewline: false
  coarseNER: "O"
  fineGrainedNER: "O"
  corefMentionIndex: 0
  nerLabelProbs: "O=0.8594029959733692"
}
token {
  word: "Along"
  pos: "IN"
  value: "Along"
  before: " "
  after: " "
  originalText: "Along"
  ner: "O"
  lemma: "along"
  beginChar: 12
  endChar: 17
  utterance: 0
  speaker: "PER0"
  tokenBeginIndex: 2
  tokenEndIndex: 3
  hasXmlC

In [ ]:
chains = ann.corefChain
chain_dict = dict()

for index, chain in enumerate(chains):
  chain_dict[index]={}
  chain_dict[index]['ref']=''
  chain_dict[index]['mentions']=[{'mentionID':mention.mentionID,
                                          'mentionType':mention.mentionType,
                                          'number':mention.number,
                                          'gender':mention.gender,
                                          'animacy':mention.animacy,
                                          'beginIndex':mention.beginIndex,
                                          'endIndex':mention.endIndex,
                                          'headIndex':mention.headIndex,
                                          'sentenceIndex':mention.sentenceIndex,
                                          'position':mention.position,
                                          'ref':'',
                                          } for mention in chain.mention ]
for k, v in chain_dict.items():
  print('key', k)
  mentions = v['mentions']
  for mention in mentions:
    words_list = ann.sentence[mention['sentenceIndex']].token[mention['beginIndex']:mention['endIndex']]
    mention['ref'] = ' '.join(t.word for t in words_list)
    if (mention['gender'] == 'MALE' or mention['gender'] == 'FEMALE') and mention['mentionType'] == 'PROPER':
      print(mention['ref'])
      print(mention['gender'])
      print(mention['mentionType'])

key 0
key 1
key 2
key 3
key 4
key 5
key 6
key 7
key 8
key 9
key 10
key 11
key 12
key 13
key 14
key 15
key 16
key 17
key 18
key 19
key 20
key 21
West Bowlby
MALE
PROPER
West Bowlby
MALE
PROPER
key 22
key 23
key 24
key 25
key 26
key 27
key 28
key 29
key 30
key 31
key 32
key 33
key 34
key 35
key 36
key 37
Jenny 's
FEMALE
PROPER
Jenny
FEMALE
PROPER
Jenny
FEMALE
PROPER
key 38
key 39
key 40
key 41
key 42
key 43
key 44
key 45
key 46
key 47
key 48
key 49
key 50
key 51
key 52
key 53
Timpany
FEMALE
PROPER
Lady Timpany
FEMALE
PROPER
Timpany
FEMALE
PROPER
Lady Timpany
FEMALE
PROPER
Lady Timpany
FEMALE
PROPER
Timpany
FEMALE
PROPER
key 54
key 55
key 56
key 57
Denis
MALE
PROPER
Denis 's
MALE
PROPER
Denis
MALE
PROPER
Mr. Scogan
MALE
PROPER
Denis
MALE
PROPER
Denis
MALE
PROPER
Denis
MALE
PROPER
Denis
MALE
PROPER
key 58
key 59
key 60
key 61
key 62
Ivor
MALE
PROPER
Ivor 's
MALE
PROPER
Ivor
MALE
PROPER
Ivor
MALE
PROPER
Ivor
MALE
PROPER
Ivor 's
MALE
PROPER
Ivor
MALE
PROPER
Ivor
MALE
PROPER
Ivor
MALE
PROPER


In [ ]:
!ps -o pid,cmd | grep java
!ls $CORENLP_HOME

   1868 /bin/bash -c ps -o pid,cmd | grep java
   1870 grep java
build.xml				  jollyday.jar
corenlp.sh				  LIBRARY-LICENSES
CoreNLP-to-HTML.xsl			  LICENSE.txt
ejml-core-0.39.jar			  Makefile
ejml-core-0.39-sources.jar		  patterns
ejml-ddense-0.39.jar			  pom-java-11.xml
ejml-ddense-0.39-sources.jar		  pom-java-17.xml
ejml-simple-0.39.jar			  pom.xml
ejml-simple-0.39-sources.jar		  protobuf-java-3.19.2.jar
input.txt				  README.txt
input.txt.out				  RESOURCE-LICENSES
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.4.0.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.4.0-javadoc.jar
javax.json.jar				  stanford-corenlp-4.4.0-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.4.0-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.jav

In [ ]:
# Iterate over all detected entity mentions
print("{:30s}\t{}".format("Mention", "Type"))

for sent in ann.sentence:
    for m in sent.mentions:
        print("{:30s}\t{}".format(m.entityMentionText, m.entityType))

A saída de streaming foi truncada nas últimas 5000 linhas.
Priscilla                     	PERSON
prophet                       	TITLE
his                           	PERSON
Barbecue-Smith                	PERSON
ten                           	NUMBER
eight                         	NUMBER
He                            	PERSON
he                            	PERSON
he                            	PERSON
his                           	PERSON
He                            	PERSON
One                           	NUMBER
Denis                         	PERSON
he                            	PERSON
Barbecue-Smith                	PERSON
Denis                         	PERSON
Barbecue-Smith                	PERSON
last winter                   	DATE
fires                         	CAUSE_OF_DEATH
his                           	PERSON
He                            	PERSON
his                           	PERSON
Denis                         	PERSON
he                            	PERSON
he                      

## Monta CSV com a lista de todos os livros presente na base, separando-os em um par de nome e título.

In [ ]:
import csv
import os

fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/fiction')
authorBookList = map(lambda file: file.rstrip('.txt').split('___', 1), fileList)
with open(os.getcwd() + '/fictionBooks.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Autor', 'Título'])
    for pair in authorBookList:
        writer.writerow(pair)

## Utilizando OpenLibrary API

In [ ]:
import json
import os
import pandas as pd
import requests
import re
from enum import Enum

class Author:
  def __init__(self, name):
    self.name = name
    self.gender = None
  
  def add_gender(self, gender):
    self.gender = gender.value

class Gender(Enum):
  FEMALE = 'F'
  MALE = 'M'
  UNKNOWN = 'U'

def get_bio(api_response):
  if 'bio' in api_response:
    if 'value' in api_response['bio']:
      return api_response['bio']['value']
    elif type(api_response['bio']) is str:
      return api_response['bio']
  return ''

maleRegex = re.compile(r'\b(he|his|him)\b')
femaleRegex = re.compile(r'\b(her|she|hers)\b')
fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/fiction')
authorBookList = map(lambda file: file.rstrip('.txt').split('___', 1), fileList)
authors = {x[0]: x for x in authorBookList}.values()
distinctauthorlist = [Author(x[0]) for x in authors]

for author in distinctauthorlist:
  response = requests.get(f'https://openlibrary.org/search/authors.json?q={author.name}').json()
  #print(response)
  if response['numFound'] > 1:
    for r in response['docs']:
      bio = get_bio(requests.get(f'https://openlibrary.org/authors/{r["key"]}.json').json())
      author.add_gender(Gender.UNKNOWN)
      if maleRegex.search(bio):
        author.add_gender(Gender.MALE)
        break
      elif femaleRegex.search(bio):
        author.add_gender(Gender.FEMALE)
        break
  elif response['numFound'] == 1:
    bio = get_bio(requests.get(f'https://openlibrary.org/authors/{response["docs"].pop()["key"]}.json').json())
    author.add_gender(Gender.UNKNOWN)
    if maleRegex.search(bio):
      author.add_gender(Gender.MALE)
    elif femaleRegex.search(bio):
      author.add_gender(Gender.FEMALE)

authorsWithGender = 0;
for author in distinctauthorlist:
  if (author.gender == 'M' or author.gender == 'F'):
    authorsWithGender += 1
  print(f'{author.name} - {author.gender}')

print(f'Total authors {len(distinctauthorlist)}')
print(f'Number of categorized authors {authorsWithGender}')
print(f'Number of uncategorized authors {len(distinctauthorlist) - authorsWithGender}')
print(f'Percentage of categorized authors {(authorsWithGender / len(distinctauthorlist)) * 100}%')


KeyboardInterrupt: ignored

## Utilizando Gender API para determinar o sexo do Personagem

In [ ]:
import json
import os
import pandas as pd
import requests
import re
from enum import Enum

class Author:
  def __init__(self, name):
    self.name = name
    self.gender = None
  
  def add_gender(self, gender):
    self.gender = gender.value

class Gender(Enum):
  FEMALE = 'F'
  MALE = 'M'
  UNKNOWN = 'U'

fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/fiction')
authorBookList = map(lambda file: file.rstrip('.txt').split('___', 1), fileList)
authors = {x[0]: x for x in authorBookList}.values()
distinctauthorlist = [Author(x[0]) for x in authors]

for author in distinctauthorlist:
  response = requests.get(f'https://gender-api.com/get?name={author.name}&key=ZoUzmKylBFxN6ww5T62G9DNfYQzVr9WRLLMt').json()
  author.add_gender(Gender[response['gender'].upper()])

authorsWithGender = 0
femaleAuthors = 0
maleAuthors = 0

for author in distinctauthorlist:
  if (author.gender == 'M' or author.gender == 'F'):
    authorsWithGender += 1
    if (author.gender == 'F'):
      femaleAuthors += 1;
    else:
      maleAuthors += 1;
  print(f'{author.name} - {author.gender}')

print(f'Total authors {len(distinctauthorlist)}')
print(f'Number of categorized authors {authorsWithGender}')
print(f'Number of uncategorized authors {len(distinctauthorlist) - authorsWithGender}')
print(f'Number of male authors {maleAuthors}')
print(f'Number of female authors {femaleAuthors}')
print(f'Percentage of categorized authors {(authorsWithGender / len(distinctauthorlist)) * 100}%')

Philip Kindred Dick - M
Rafael Sabatini - M
R M Ballantyne - M
Robert Louis Stevenson - M
Rudyard Kipling - M
Sinclair Lewis - M
Sir Arthur Conan Doyle - M
Sir Walter Scott - M
Stephen Leacock - M
Thomas Carlyle - M
Thomas Hardy - M
Thornton Waldo Burgess - M
Virginia Woolf - F
Washington Irving - M
Walter de la Mare - M
Wilkie Collins - M
William Dean Howells - M
William Makepeace Thackeray - M
William Somerset Maugham - M
Winston Churchill - M
Zane Grey - M
Harold Bindloss - M
Henry James - M
Henry Rider Haggard - M
Herbert George Wells - M
Herman Melville - M
Howard Pyle - M
Jack London - M
Isaac Asimov - M
Jacob Abbott - M
James Fenimore Cooper - M
James Joyce - M
James Matthew Barrie - M
James Otis - M
Jane Austen - F
John Bunyan - M
John Galsworthy - M
John Keats - M
Joseph Conrad - M
Lewis Carroll - M
Louisa May Alcott - F
Lucy Maud Montgomery - F
Mark Twain - M
Lyman Frank Baum - M
Mary Shelley - F
Nathaniel Hawthorne - M
Mary Stewart Daggett - F
O Henry - M
Oscar Wilde - M
And